# Assignment Week 1 

## 1. MP (Markov Process)

### Definition

The Markov Process (MP) satisfies the Markov property:
$$
P(S_{t+1}|S_{t}, S_{t-1}, S_{t-2}\dots) = P(S_{t+1}|S_{t})
$$

So, in MDP process, we can represent the transformission probability as: 
$$
P_{ss'} = P(S_{t+1} = s' |S_{t} = s)
$$

If we consider discrete time model, we call it a Markov Chain where probability varies with time:
$$
P^t_{ss'} = P^t(S_{t+1} = s' |S_{t} = s)
$$

In a lot of cases we only consider a time-homogeneous(or time-independent) transition probablity.

Now let's consider the student problem shown in David Silver's slides.

In [4]:
import numpy as np
from scipy.linalg import eig
from typing import Mapping,Sequence,Set
import sys
sys.path.append('Code/Processes/Markov')
sys.path.append('Code/Utils')
from Markov_Functions import verify_mp
from Generic_TypeVars import S
from Standard_TypeVars import SSf

class MP:
    # The Mapping type indicates a dict[state, dict[state, number]]
    # self.transition_graph is a dictionary
    def __init__(self, graph: SSf):
        if verify_mp(graph):
            self.state: Sequence[S] = list(graph.keys())
            self.transition_graph: Mapping[S, Mapping[S, float]] = graph
            self.transition_matrix: np.array = self.find_transition_matrix()
        else:
            raise ValueError
    
    def find_transition_matrix(self)->np.array:
        '''Transition Matrix'''
        num: int = len(self.state)
        transition_matrix = np.zeros((num,num))
        for i in range(num):
            s=self.state[i]
            for j in range(num):
                s_prime=self.state[j]
                if s_prime in self.transition_graph[s].keys():
                    transition_matrix[i,j]=self.transition_graph[s][s_prime]
        return transition_matrix    
   
    def find_transition_probability(self,cur_state:S,next_state:S)->float:
        '''Get specific transition Probabilities between two states'''
        if cur_state in self.state and next_state in self.state:
            if next_state in self.transition_graph[cur_state].keys():
                
                return self.transition_graph[cur_state][next_state]
            else:
                return 0.0
        else: 
            raise ValueError
            
    def find_stationary_distribution(self) -> Mapping[S, Mapping[S, float]]:
        '''Stationary Probabilities'''
        # We use the eigen-value approach
        eig_val, eig_vec_right = eig(self.transition_matrix.T)
        eig_vec_left = eig_vec_right[:,np.isclose(eig_val, 1)]
        eig_vec = eig_vec_left[:,0]
        stationary = eig_vec/eig_vec.sum()
        stationary = stationary.real
        return {s: stationary[i] for i, s in enumerate(self.state)}
    
    def check_stationary_distribution(self) -> np.array:
        return NotImplementedError
    
    def find_transition_graph(self) -> Mapping[S, Mapping[S, float]]:
        '''Current Graph'''
        return self.transition_graph
    
    def find_all_states(self) -> Sequence[S]:
        '''Current State'''
        return self.state    
    
    def find_sink_states(self) -> Set[S]:
        return {k for k, v in self.transition_graph.items()
                if len(v) == 1 and k in v.keys()}

if __name__=='__main__':

    student = {'C1':{'C2': 0.5, 'Facebook': 0.5},
               'C2':{'C3': 0.8, 'Sleep': 0.2},
               'C3':{'Pass': 0.6, 'Pub': 0.4},
               'Pass':{'Sleep': 1.0},
               'Pub':{'C1': 0.2, 'C2': 0.4, 'C3': 0.4},
               'Facebook':{'C1': 0.1, 'Facebook': 0.9},
               'Sleep':{'Sleep': 1.0}}
    
    test_mp=MP(student)
    print('Find transition matrix :\n',test_mp.find_transition_matrix(),'\n')
    print('Find stationary distribution:\n',test_mp.find_stationary_distribution(),'\n')
    print('Find all states:\n',test_mp.find_all_states(),'\n')
    print('Find sink state:\n',test_mp.find_sink_states(),'\n')

Find transition matrix :
 [[0.  0.5 0.  0.  0.  0.5 0. ]
 [0.  0.  0.8 0.  0.  0.  0.2]
 [0.  0.  0.  0.6 0.4 0.  0. ]
 [0.  0.  0.  0.  0.  0.  1. ]
 [0.2 0.4 0.4 0.  0.  0.  0. ]
 [0.1 0.  0.  0.  0.  0.9 0. ]
 [0.  0.  0.  0.  0.  0.  1. ]] 

Find stationary distribution:
 {'C1': 0.0, 'C2': 0.0, 'C3': 0.0, 'Pass': 0.0, 'Pub': 0.0, 'Facebook': 0.0, 'Sleep': 1.0} 

Find all states:
 ['C1', 'C2', 'C3', 'Pass', 'Pub', 'Facebook', 'Sleep'] 

Find sink state:
 {'Sleep'} 




## 2. MRP (Markov Reward Process)

### Definition

Based on the MP property, the  Markov Reward Proces (MRP) takes the Reward $R_t $, as well as the Discount Factor $\gamma \in [0,1] $ into account. We are going to model the transmission of  State $S$. We still need State $S$ and Transition Probability $P$, and also the reward function. However, there are several definitions about reward function:

$$\text{ David Silver} : R_{s}=E[R_{t+1} \mid S_t=s]$$

which is an expected value of $s^{'}$, or mathematically

$$ R_{s} = \sum_{s' \in S}^{}{P_{ss'}R_{ss'}}$$



### Return 

We also introduce a return $G_t$ which can be calculated as:

$$G_t=R_{t+1}+\gamma R_{t+2}+...=\sum_{k=0}^{\infty}\gamma^k R_{t+k+1}$$

The discount factor should be chosen based on certain situations, rather than need of convergence!

### Value Function

Based on definitions and functions of MRP, we can come up with  Value Function $(V(s))$, which is a function of state $S$: 
$$v(s)=E[G_t \mid S_t=s]$$ 

$v(s)$ has this explanation: the expected return of state s will be $v(s)$. Expand the expected value function: $$\begin{split} v(s) & =  E[G_t \mid S_t=s] \\ & =  E[R_{t+1}+ \gamma R_{t+2} + \gamma^2 R_{t+3}+... \mid S_t=s] \\ & =  E[R_{t+1}+ \gamma G_{t+1} \mid S_t=s] \\ & =  E[R_{t+1}+ \gamma v(S_{t+1}) \mid S_t=s] \\ & =   R_{s} + \gamma \sum_{s^{'} \in  S} P_{ss^{'}}v(s^{'}) \end{split}$$ So we can see, the value of each state is equal to the conditional expected value of immediate reward $R_{s}$ , plus the dicounted value of successor state $\gamma v(S_{t+1})$, based on that given state $S_t=s$.

### Bellman Equation

So, given an MRP, all states have their unique value functions. Obviously, it depends on the transition probability and reward in each state.

Then we can find out such equation, called Bellman Equation:
$$v(s)= R_{s} + \gamma \sum_{s^{'} \in  S}  P_{ss^{'}}v(s^{'})$$

Further more, the matrix form can be written as:
$$v=(I- \gamma  P)^{-1}  R$$

Let's still consider the student problem shown in David Silver's slides.

In [7]:
### MRP add two more properties on MP
#in this section, the default reward function based on david silver definition
import numpy as np
import sys
sys.path.append('Code/Utils')
sys.path.append('Code/Processes/Markov')
from MP import MP
from typing import Mapping
from Generic_TypeVars import S
from Standard_TypeVars import STSff
from General_Utils import zip_dict_of_tuple

class MRP(MP):
    
    def __init__(self, mrp_graph:STSff, gamma:float):
        # Super Class from MP
        mp_graph, reward_graph = zip_dict_of_tuple(mrp_graph)
        super().__init__(mp_graph)                                        
        self.reward_graph: Mapping[S, float] = reward_graph
        # Gamma is the discount factor
        self.gamma: float = gamma
        self.reward_vector=self.find_reward_vector()
        
    def find_reward(self) -> Mapping[S, float]:
        return self.reward_graph

    def find_reward_vector(self) -> np.array:
        return np.array([self.reward_graph[i] for i in self.state])
    
    def find_value_function_vector(self) -> np.array:
        # Bellman Equation in Matrix Form
        # Blows up when gamma = 1
        return np.linalg.inv(np.eye(len(self.state)) - self.gamma * self.transition_matrix
        ).dot(self.find_reward_vector())
    

if __name__ == '__main__':
    student = {'C1':({'C2': 0.5, 'Facebook': 0.5}, -2.0),
               'C2':({'C3': 0.8, 'Sleep': 0.2}, -2.0),
               'C3':({'Pass': 0.6, 'Pub': 0.4}, -2.0),
               'Pass':({'Sleep': 1.0}, 10.0),
               'Pub':({'C1': 0.2, 'C2': 0.4, 'C3': 0.4}, 1.0),
               'Facebook':({'C1': 0.1, 'Facebook': 0.9}, -1.0),
               'Sleep':({'Sleep': 1.0}, 0.0)}
    gamma = 0.9999999
    test_mrp = MRP(student, gamma)
    print('Get value function vector for gamma = 1:\n',test_mrp.find_value_function_vector(),'\n')
    print('Find reward vector:\n', test_mrp.find_reward_vector())   

Get value function vector for gamma = 1:
 [-12.54318511   1.45679124   4.32098948  10.           0.80247529
 -22.54316356   0.        ] 

Find reward vector:
 [-2. -2. -2. 10.  1. -1.  0.]
